In [2]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=47aa101a4a608ff2f3b3101f62ab94020369f0538d443e536a9745ba0897e014
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [39]:
%pip install -qU langchain-openai

In [1]:
from langchain_openai import AzureOpenAI

In [12]:
# Import Azure OpenAI

from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
    temperature=0,
    api_key="api",
    api_version="2024-02-01",
    azure_endpoint="https://dono-rag-demo-resource-instance.openai.azure.com",
    model="GPT_35_TURBO_DEMO_RAG_DEPLOYMENT_DONO"
)

In [3]:
from langchain_community.document_loaders import WikipediaLoader

In [4]:
link="https://en.wikipedia.org/wiki/FIFA_World_Cup#Results"
docs = WikipediaLoader(query=link, load_max_docs=5).load()
len(docs)

5

In [5]:
# Extract the text from the website data document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_content = ""
for page in docs:
    text_content += page.page_content

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=512, chunk_overlap=0)

doc = text_splitter.split_documents(docs)


In [26]:
# pip install sentence_transformers

In [25]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
# Save to disk
vectorstore = Chroma.from_documents(
                     documents=doc,                 # Data
                     embedding=embedding_function,    # Embedding model
                     persist_directory="./chroma_db" # Directory to save data
                     )

In [28]:
# Load from disk
vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",       # Directory of db
                        embedding_function=embedding_function   # Embedding model
                   )

In [29]:
retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 5})

In [34]:
from langchain import PromptTemplate
llm_prompt_template = """You are an assistant for question-answering tasks.
Use the following context to answer the question.
I have give you a data of 5 years football match,You have to answer in few word of the querry.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

In [38]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser


In [39]:
# Combine data from documents to readable string format.

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    | model
    | StrOutputParser()
)

In [40]:
rag_chain.invoke("Who won n 2015 FIFA World Cup?")

'The 2015 FIFA World Cup was won by the United States.'